[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

In [1]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Overview of the most serious COVID-19 cases

This graph shows the daily count of COVID-19 hospital cases, COVID-19 patients on ventilators and new COVID-19 related deaths. You can select the metrics to be displayed.
* Total number of confirmed COVID-19 patients in hospital on the reporting date.
* Total number of mechanical ventilation beds that are occupied by COVID-19 patients on the reporting date.
* Total number of people who had a positive test result for COVID-19 and died within 28 days, reported on the date of death.

In [3]:
# the wrangle_data function will be called on the JSON files when the dashboard first loads
# and again when the data is refreshed through the API
def wrangle_timeseries_data(rawData, columnList):
    """ Parameters: raw timeseries data from json or API call and a list of columns. Returns a populated dataframe. """

    # pull a sorted list of dates from rawData
    rawDataList = rawData['data']
    dates = []
    for item in rawDataList:
        dates.append(item['date'])
    dates.sort()

    # define a function to convert our date into a panda date object
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    # use the panda date_range method to create a date range from the first item in our sorted date list to the last
    # this forms our index (x-axis)
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    dateRange=pd.date_range(startdate, enddate, freq='D')
    
    # create empty dataframe
    newDF=pd.DataFrame(index=dateRange, columns=columnList)
    
    # populate dataframe
    for item in rawDataList:
        date=parse_date(item['date'])  # convert rawData dates into our datetime objects
        for col in columnList:
            if pd.isna(newDF.loc[date, col]):  # check that the cell in the dataframe is currently NaN
                if item[col] != None:  # if the raw data is not None
                    newDF.loc[date, col] = float(item[col])  # populate the cell with the value at key=col
                else:
                    newDF.loc[date, col] = 0.0  # else replace None with zero
    newDF.fillna(0.0, inplace=True) # fill any remaining NAs in the dataframe with zero 
       
    # return the populated dataframe
    return newDF

In [14]:
# call the function directly on the JSON data when the dashboard starts
# load initial data
with open("hospitalTimeseries.json", "rt") as INFILE:
    hospitalJSON=json.load(INFILE)

# call wrangle_data function. Parameters are my json data and the list of columns. Returns a populated dataframe
hospitalDF = wrangle_timeseries_data(hospitalJSON, ['hospitalCases', 'ventilators','newDeaths'])

In [5]:
# This function is called when the button is clicked.
# the code accesses the API and returns fresh raw data
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
        'areaType=overview',
    ]
    structure = {
        "date": "date",
        "hospitalCases": "hospitalCases",
        "ventilators": "covidOccupiedMVBeds",
        "newDeaths": "newDeaths28DaysByDeathDate"
    }

    api = Cov19API(filters=filters, structure=structure)
    hospitalTimeseries=api.get_json()
    # return raw data read from the API
    return hospitalTimeseries

In [6]:
# this function is called in the api_button_callback
# a function to simulate interaction with the widget.
def refresh_graph(selectWidget):
    """ We change the value of the widget in order to force a redraw of the graph. """
    current = selectWidget.value
    if current == selectWidget.options:
        other = selectWidget.options[0:1]
    else:
        other = selectWidget.options
    selectWidget.value=other # forces the redraw
    selectWidget.value=current # now we can change it back. Comment this out to check if the function is working

In [10]:
# Get fresh data from the API.
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    ## error handling: the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached
    try:
        apidata=access_api()
        # wrangle the data into a dataframe
        # and overwrite the (global) hospitalDF variable for plotting
        global hospitalDF
        hospitalDF = wrangle_timeseries_data(apidata, ['hospitalCases', 'ventilators','newDeaths'])
        # call the refresh_graph function defined above which simulates interaction with the widget.
        refresh_graph(selectColumns)
        # update the button icon/text for a success scenario
        apibutton.description="Data refreshed"
        button_style='success',
        apibutton.icon="check"
        apibutton.disabled=True
    except:
        # update the button icon/text for an error scenario
        apibutton.description="Data unavailable"
        button_style='danger',
        apibutton.icon="exclamation-triangle"
        apibutton.disabled=True

# initial api button
apibutton=wdg.Button(
    description='Refresh PHE data', 
    disabled=False,
    button_style='info',
    tooltip="Access the latest data from Public Health England",
    icon='download'
)

# register the button callback function with the button, so the api is refreshed when the button is clicked
apibutton.on_click(api_button_callback) 

# display the button
display(apibutton)

Button(button_style='info', description='Refresh PHE data', icon='download', style=ButtonStyle(), tooltip='Acc…

In [15]:
# this function enables interactive plotting
def plot_graph(gcols):
    ncols=len(gcols)
    if ncols>0:
        global hospitalDF
        hospitalDF[list(gcols)].plot(title = 'Overview of the most serious COVID-19 cases')
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    # print the timestamp of the most recent date in the dataframe
    print("Latest data as of: ", end='')
    print(hospitalDF.index[-1].strftime('%d-%b-%Y'))

selectColumns=wdg.SelectMultiple(
    options=['hospitalCases', 'ventilators', 'newDeaths'], # possible values
    value=['hospitalCases', 'ventilators', 'newDeaths'], # default value
    rows=3,
    description='Select:',
    disabled=False
)

# create interactive graph
graph=wdg.interactive_output(plot_graph, {'gcols': selectColumns})

# display the selection box and the graph together (horizontally)
form=wdg.HBox([selectColumns, graph])
display(form)

**Author and Copyright Notice** Amy Shaw (2020). Template by Fabrizio Smeraldi (2020).
*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*